<a href="https://colab.research.google.com/github/kurtvalcorza/notebooks/blob/main/speech_to_text_diarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```markdown
# Audio Diarization with OpenAI's `gpt-4o-transcribe-diarize`

This notebook demonstrates how to perform audio diarization using OpenAI's `gpt-4o-transcribe-diarize` model. It takes an audio file as input, processes it in chunks, and generates a diarized transcript, identifying different speakers in the audio.

## Features

*   **Audio Diarization:** Identifies and separates speakers in an audio file.
*   **Chunking:** Processes large audio files by splitting them into smaller chunks to meet API limits.
*   **Retry Mechanism:** Includes a retry logic for API calls to handle transient errors.
*   **Speaker Recognition (Optional):** Supports providing known speaker references for improved diarization.

## Setup

1.  **OpenAI API Key:**
    *   Obtain an OpenAI API key from the [OpenAI platform](https://platform.openai.com/).
    *   In Google Colab, go to the "🔑" (Secrets) icon on the left panel.
    *   Add a new secret named `OPENAI_API_KEY` and paste your OpenAI API key as the value.

2.  **Install Dependencies:**
    The notebook automatically installs the required Python packages (`openai`, `pydub`) and `ffmpeg`.

## Usage

1.  **Upload Audio File:**
    *   Upload your audio file (e.g., `audio.wav`, `meeting.mp3`) to the Colab environment. You can drag and drop it into the file browser on the left panel or use the code cells to upload.

2.  **Configure Input File:**
    *   Modify the `input_file` variable in the code to point to your uploaded audio file.
    *   _Optional:_ If you have a reference audio file for a known speaker, upload it and update `known_speaker_ref_file` and `known_speaker_name` accordingly.

3.  **Run All Cells:**
    *   Execute all cells in the notebook. The script will process the audio, perform diarization, and save the transcript to a text file (e.g., `audio_diarized_output.txt`).

4.  **View Output:**
    *   The diarized transcript will be saved in the same directory as a `.txt` file, which you can then download or view directly in Colab.

## Output Format

The output file will contain lines in the format:

```
[start_time-end_time] Speaker_Identifier: Transcribed Text
```

Example:

```
[0.00-2.50] Speaker 1: Hello, how are you today?
[2.50-5.10] Speaker 2: I'm doing great, thanks for asking!
```
```

In [ ]:
!pip install -U openai pydub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1


In [ ]:
!apt-get install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
import os
import time
import base64
from openai import OpenAI
from pydub import AudioSegment
from google.colab import userdata

# Set the API key for the OpenAI package
client = OpenAI(
    api_key=userdata.get("OPENAI_API_KEY"), # Use Colab's Secrets manager
)

# Input and output set to your audio (e.g., 'meeting.mp3')
input_file = "/content/audio.wav"  #@param {type: "string"}
output_file = os.path.splitext(input_file)[0] + "_diarized_output.txt"

# Diarization configuration
NUM_SPEAKERS = None  # set to an int if you know the count, else leave as None
known_speaker_name = "agent"
known_speaker_ref_file = "agent.wav"  # used if present

# API chunk limit: keep under 1400 seconds; use a safety buffer
# Also control file size by reducing duration and bitrate.
MAX_DURATION_MS = 600 * 1000  # 10 minutes per chunk
MAX_RETRIES = 5
INITIAL_DELAY = 1.0

def to_data_url(path: str) -> str:
    with open(path, "rb") as fh:
        return "data:audio/wav;base64," + base64.b64encode(fh.read()).decode("utf-8")

def build_extra_body():
    eb = {}
    if os.path.exists(known_speaker_ref_file):
        eb.update({
            "known_speaker_names": [known_speaker_name],
            "known_speaker_references": [to_data_url(known_speaker_ref_file)],
        })
    if isinstance(NUM_SPEAKERS, int) and NUM_SPEAKERS > 0:
        eb["num_speakers"] = NUM_SPEAKERS
    return eb

def process_chunk(chunk_path, chunk_start_ms, extra_body, attempt=0):
    if attempt >= MAX_RETRIES:
        print(f"Failed after {MAX_RETRIES} attempts: {chunk_path}")
        return []
    try:
        with open(chunk_path, "rb") as audio_file:
            print(f"Transcribing chunk at {chunk_start_ms/1000:.2f}s: {os.path.basename(chunk_path)} (attempt {attempt+1})")
            resp = client.audio.transcriptions.create(
                model="gpt-4o-transcribe-diarize",
                file=audio_file,
                response_format="diarized_json",
                chunking_strategy="auto",
                extra_body=extra_body,
            )
        # Adjust timestamps to absolute
        adj = []
        offset = chunk_start_ms / 1000.0
        for seg in resp.segments:
            adj.append({
                'speaker': seg.speaker,
                'text': seg.text,
                'start': seg.start + offset,
                'end': seg.end + offset,
            })
        return adj
    except Exception as e:
        em = str(e).lower()
        print(f"Error on chunk {chunk_path}: {e}")
        if 'corrupted' in em or 'unsupported' in em or "param': 'file'" in em:
            # Fallback: re-encode to 16kHz mono WAV and retry once for this attempt level
            try:
                from pydub import AudioSegment as _AS
                wav_path = chunk_path + '.wav'
                _AS.from_file(chunk_path).set_channels(1).set_frame_rate(16000).export(wav_path, format='wav')
                with open(wav_path, 'rb') as audio_file:
                    resp = client.audio.transcriptions.create(
                        model="gpt-4o-transcribe-diarize",
                        file=audio_file,
                        response_format="diarized_json",
                        chunking_strategy="auto",
                        extra_body=extra_body,
                    )
                try:
                    os.remove(wav_path)
                except OSError:
                    pass
                adj = []
                offset = chunk_start_ms / 1000.0
                for seg in resp.segments:
                    adj.append({
                        'speaker': seg.speaker,
                        'text': seg.text,
                        'start': seg.start + offset,
                        'end': seg.end + offset,
                    })
                return adj
            except Exception as _e:
                print(f"Fallback WAV attempt failed: {_e}")
                # continue to generic retry below
        if 'rate limit' in em or 'server error' in em:
            delay = INITIAL_DELAY * (2 ** attempt)
            print(f"Retrying in {delay:.2f}s...")
            time.sleep(delay)
            return process_chunk(chunk_path, chunk_start_ms, extra_body, attempt + 1)
        raise

# Load audio and split as needed
if not os.path.exists(input_file):
    raise FileNotFoundError(f"Input audio not found: {input_file}")

print(f"Loading audio: {input_file}")
audio = AudioSegment.from_file(input_file)
duration_sec = len(audio) / 1000.0
print(f"Audio duration: {duration_sec:.2f}s")

start_ms = 0
chunk_idx = 0
all_segments = []
temp_files = []
extra_body = build_extra_body()

while start_ms < len(audio):
    end_ms = min(start_ms + MAX_DURATION_MS, len(audio))
    chunk = audio[start_ms:end_ms]
    # Re-encode to mono 16kHz to ensure compatibility and size reduction
    chunk = chunk.set_channels(1).set_frame_rate(16000)
    # Export WAV (PCM 16kHz mono) to avoid mp3 encoder issues
    tmp_name = f"temp_chunk_{chunk_idx}.wav"
    chunk.export(tmp_name, format="wav")
    temp_files.append(tmp_name)

    segs = process_chunk(tmp_name, start_ms, extra_body)
    for s in segs:
        all_segments.append(s)

    start_ms = end_ms
    chunk_idx += 1

# Save diarized output
if all_segments:
    with open(output_file, "w", encoding="utf-8") as f:
        for s in all_segments:
            f.write(f"[{s['start']:.2f}-{s['end']:.2f}] {s['speaker']}: {s['text']}\n")
    print(f"Diarized transcript saved to {output_file}")
else:
    print("No segments returned.")

# Cleanup
for fp in temp_files:
    try:
        os.remove(fp)
    except OSError as e:
        print(f"Warning: could not delete {fp}: {e}")

Loading audio: /content/audio.wav
Audio duration: 18.36s
Transcribing chunk at 0.00s: temp_chunk_0.wav (attempt 1)
Diarized transcript saved to /content/audio_diarized_output.txt
